<a href="https://colab.research.google.com/github/eugin0901/eugin0901/blob/main/16%2B16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pandas

In [3]:
pip install scikit-learn

In [4]:
pip install openpyxl

In [5]:
pip install soynlp kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.3 MB/s eta 0:00:00
  Created wheel for kiwipiepy_model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818026 sha256=b0793784077e842870f171c9654ee2824931ded06f173f352dd3b4f1be6bee7f
  Stored in directory: /root/.cache/pip/wheels/ca/c8/52/3a539d6e9065b191fe1c215e0203dcc3e00601c0e3d3d39824
Successfully built kiwipiepy_model


In [6]:
# 필요한 라이브러리 설치 및 불러오기
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re

In [8]:
# 데이터 로드
data = pd.read_excel("final_datalist.xlsx")

In [10]:
# 시기 분류 함수
def classify_period(date):
    if pd.isnull(date):
        return "알 수 없음"
    elif date.year <= 2014:
        return "1기"
    elif date.year <= 2019:
        return "2기"
    else:
        return "3기"

# 날짜 열을 datetime 형식으로 변환
data['일자'] = pd.to_datetime(data['일자'], format='%Y%m%d', errors='coerce')

# 시기 분류 열 추가
data['시기'] = data['일자'].apply(classify_period)

In [11]:
# 통합 텍스트 생성
data['통합 텍스트'] = data['키워드'].fillna('') + ' ' + data['특성추출(가중치순 상위 50개)'].fillna('')

In [26]:
# 전처리 함수 정의
def preprocess_text(text):
    try:
        # 한글만 남기고 정규화
        text = re.sub(r'[^가-힣\s,]', '', text)  # 쉼표(,)는 유지
        # 쉼표를 기준으로 분리
        words = text.split(',')  # 쉼표 기준으로 분리
        # 두 글자 이상의 단어만 추출
        words = [word.strip() for word in words if len(word.strip()) > 1]  # 공백 제거 후 길이 체크
        return ' '.join(words)  # 다시 문자열로 반환 (단어들을 공백으로 연결)
    except Exception as e:
        print(f"전처리 중 오류 발생: {e}, 입력 데이터: {text}")
        return ''

# 전처리된 텍스트 생성
data['전처리된 텍스트'] = data['통합 텍스트'].apply(preprocess_text)

In [27]:
# 시기별 데이터 분할
periods = {
    "1기": data[data['시기'] == "1기"],
    "2기": data[data['시기'] == "2기"],
    "3기": data[data['시기'] == "3기"]
}

# 각 시기별 데이터 크기 확인
for period, df in periods.items():
    print(f"{period} 데이터 개수: {len(df)}")

1기 데이터 개수: 117
2기 데이터 개수: 364
3기 데이터 개수: 887


In [33]:
# LDA 실행 함수 정의
def run_lda(period_name, df, num_topics=3):
    print(f"\n[{period_name}] LDA 실행 중...")

    # 전처리된 텍스트 추출
    texts = df['전처리된 텍스트'].tolist()

    # 빈 텍스트 제거
    texts = [text for text in texts if text.strip()]
    if len(texts) == 0:
        print(f"[{period_name}] 전처리된 텍스트가 없습니다. LDA 실행을 건너뜁니다.")
        return None, None, None

    # Bag of Words 모델 생성
    vectorizer = CountVectorizer(max_df=1.0, min_df=1)  # min_df와 max_df 조정
    dtm = vectorizer.fit_transform(texts)

    # LDA 모델 훈련
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda_model.fit(dtm)

    # 토픽 출력
    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda_model.components_):
        print(f"토픽 {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-11:-1]]))

    return lda_model, dtm, vectorizer

# 시기별로 LDA 실행 및 결과 저장
lda_results = {}
for period, df in periods.items():
    if len(df) > 0:  # 데이터가 있는 경우에만 실행
        lda_results[period] = run_lda(period, df)


[1기] LDA 실행 중...
토픽 1:
치과 산업 디지털 기업 지원 의료 바텍 시장 융합 대표
토픽 2:
임플란트 치아 치과 치료 환자 수술 시술 원장 보철물 가능
토픽 3:
치과 의료 디오 장비 디지털 시장 병원 대표 제품 임플란트

[2기] LDA 실행 중...
토픽 1:
임플란트 디지털 치아 환자 시술 치료 수술 치과 교정 진행
토픽 2:
치과 디지털 산업 의료 치의학 기업 제품 시장 세계 제작
토픽 3:
시장 임플란트 오스템임플란트 중국 매출 치과 디지털 제품 성장 덴티움

[3기] LDA 실행 중...
토픽 1:
임플란트 치아 치료 디지털 수술 환자 구강 교정 치과 식립
토픽 2:
시장 투자 인수 기업 매출 중국 메디트 실적 대비 성장
토픽 3:
치과 디지털 시장 임플란트 기업 의료 제품 솔루션 대표 오스템임플란트


In [36]:
# 시기별로 LDA 실행 및 결과 저장
lda_results = {}
for period, df in periods.items():
    if len(df) > 0:  # 데이터가 있는 경우에만 실행
        lda_results[period] = run_lda(period, df)

# 주요 토픽별 문서 수 확인
for period, (lda_model, dtm, vectorizer) in lda_results.items():
    print(f"\n[{period}] 주요 토픽별 문서 수:")
    topic_assignments = lda_model.transform(dtm).argmax(axis=1)  # 각 문서의 가장 높은 확률을 가진 토픽 번호
    topic_counts = pd.Series(topic_assignments).value_counts()  # 토픽별 문서 수 계산
    print(topic_counts)

    # 각 문서에 토픽 번호 추가 (데이터프레임에 열 추가)
    periods[period]['토픽 번호'] = topic_assignments

# 주요 토픽별 문서 수를 포함한 데이터를 엑셀로 저장
def save_to_excel(periods, output_file="lda_results.xlsx"):
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for period, df in periods.items():
            if len(df) > 0:  # 데이터가 있는 경우에만 저장
                df.to_excel(writer, sheet_name=period, index=False)  # 시트 이름은 시기 이름
                print(f"[{period}] 데이터 저장 완료.")
    print(f"모든 데이터를 {output_file}에 저장 완료.")

# 엑셀 저장 실행
output_file = "lda_results202501292056.xlsx"
save_to_excel(periods, output_file=output_file)


[1기] LDA 실행 중...
토픽 1:
치과 산업 디지털 기업 지원 의료 바텍 시장 융합 대표
토픽 2:
임플란트 치아 치과 치료 환자 수술 시술 원장 보철물 가능
토픽 3:
치과 의료 디오 장비 디지털 시장 병원 대표 제품 임플란트

[2기] LDA 실행 중...
토픽 1:
임플란트 디지털 치아 환자 시술 치료 수술 치과 교정 진행
토픽 2:
치과 디지털 산업 의료 치의학 기업 제품 시장 세계 제작
토픽 3:
시장 임플란트 오스템임플란트 중국 매출 치과 디지털 제품 성장 덴티움

[3기] LDA 실행 중...
토픽 1:
임플란트 치아 치료 디지털 수술 환자 구강 교정 치과 식립
토픽 2:
시장 투자 인수 기업 매출 중국 메디트 실적 대비 성장
토픽 3:
치과 디지털 시장 임플란트 기업 의료 제품 솔루션 대표 오스템임플란트

[1기] 주요 토픽별 문서 수:
1    49
2    42
0    26
Name: count, dtype: int64

[2기] 주요 토픽별 문서 수:
2    134
1    117
0    113
Name: count, dtype: int64

[3기] 주요 토픽별 문서 수:


<ipython-input-36-2eaf72554b9f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  periods[period]['토픽 번호'] = topic_assignments
<ipython-input-36-2eaf72554b9f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  periods[period]['토픽 번호'] = topic_assignments


2    389
1    368
0    130
Name: count, dtype: int64
[1기] 데이터 저장 완료.
[2기] 데이터 저장 완료.


<ipython-input-36-2eaf72554b9f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  periods[period]['토픽 번호'] = topic_assignments


[3기] 데이터 저장 완료.
모든 데이터를 lda_results202501292056.xlsx에 저장 완료.


In [37]:
# 토픽별 주요 단어 출력 함수
def print_topics(model, vectorizer, top_n=10):
    feature_names = vectorizer.get_feature_names_out()  # 단어 목록 추출
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n토픽 {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-top_n - 1:-1]]))

# LDA 모델 실행 후 토픽 출력
print('1기')
lda_model, dtm, vectorizer = lda_results['1기']  # 예: '1기' 데이터
print_topics(lda_model, vectorizer, top_n=10)


# LDA 모델 실행 후 토픽 출력
print('2기')
lda_model, dtm, vectorizer = lda_results['2기']  # 예: '2기' 데이터
print_topics(lda_model, vectorizer, top_n=10)

# LDA 모델 실행 후 토픽 출력
print('3기')
lda_model, dtm, vectorizer = lda_results['3기']  # 예: '3기' 데이터
print_topics(lda_model, vectorizer, top_n=10)

1기

토픽 1:
치과 산업 디지털 기업 지원 의료 바텍 시장 융합 대표

토픽 2:
임플란트 치아 치과 치료 환자 수술 시술 원장 보철물 가능

토픽 3:
치과 의료 디오 장비 디지털 시장 병원 대표 제품 임플란트
2기

토픽 1:
임플란트 디지털 치아 환자 시술 치료 수술 치과 교정 진행

토픽 2:
치과 디지털 산업 의료 치의학 기업 제품 시장 세계 제작

토픽 3:
시장 임플란트 오스템임플란트 중국 매출 치과 디지털 제품 성장 덴티움
3기

토픽 1:
임플란트 치아 치료 디지털 수술 환자 구강 교정 치과 식립

토픽 2:
시장 투자 인수 기업 매출 중국 메디트 실적 대비 성장

토픽 3:
치과 디지털 시장 임플란트 기업 의료 제품 솔루션 대표 오스템임플란트


In [49]:
!pip install --upgrade pyLDAvis

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [53]:
import pyLDAvis
import os

# 시각화 결과를 저장할 디렉토리 생성
output_dir = "lda_visualizations"
os.makedirs(output_dir, exist_ok=True)  # 디렉토리가 없으면 생성

# 시각화 및 HTML 저장 함수
def visualize_lda(lda_model, dtm, vectorizer, period_name):
    try:
        # pyLDAvis 준비
        vis = pyLDAvis.prepare(
            topic_term_dists=lda_model.components_,
            doc_topic_dists=lda_model.transform(dtm),
            doc_lengths=[sum(doc) for doc in dtm.toarray()],
            vocab=vectorizer.get_feature_names_out(),
            term_frequency=dtm.toarray().sum(axis=0),
            mds='tsne'
        )

        # HTML 파일 저장 경로
        html_file = os.path.join(output_dir, f"{period_name}_lda_visualization.html")

        # HTML 저장
        pyLDAvis.save_html(vis, html_file)
        print(f"[{period_name}] LDA 시각화 결과를 {html_file}에 저장했습니다.")
    except Exception as e:
        print(f"[{period_name}] 시각화 중 오류 발생: {e}")

# 시기별로 LDA 시각화 실행 및 저장
for period, result in lda_results.items():
    if result is not None:
        lda_model, dtm, vectorizer = result
        visualize_lda(lda_model, dtm, vectorizer, period)



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[1기] LDA 시각화 결과를 lda_visualizations/1기_lda_visualization.html에 저장했습니다.
[2기] LDA 시각화 결과를 lda_visualizations/2기_lda_visualization.html에 저장했습니다.
[3기] LDA 시각화 결과를 lda_visualizations/3기_lda_visualization.html에 저장했습니다.
